In [1]:
import sys
from pathlib import Path
sys.path = list(set(sys.path + ["../python"]))

In [2]:
import numpy
import pandas as pd
from gaussian_system import System, time_matrix, estimate_log_marginal

In [3]:
path = Path.cwd().parent / "plots"
assert path.exists()
path

PosixPath('/Users/mr/Git/Gillespie/plots')

In [4]:
system = System(lamda = 0.005, kappa = 0.25, rho = 0.01, mu = 0.01)
traj_len = 500

In [5]:
def marginal_entropy_estimate(system, dim, delta_t, num_x, num_s):
    _, p_x = estimate_log_marginal(num_x, num_s, system, time_matrix(dim, delta_t))
    return pd.DataFrame(
        {
            "dim": dim,
            "delta_t": delta_t,
            "num_responses": 1,
            "num_signals": num_s,
            "marginal_entropy": -p_x,
        }
    )

In [6]:
def generate_data():
    estimates = (
        marginal_entropy_estimate(system, dim, traj_len / dim, 400, 550)
        for dim in numpy.linspace(100, 400, 6).astype(int)
    )
    pd.concat(estimates, ignore_index=True).to_json(
        path / "monte-carlo" / "data.json", orient="table"
    )

In [7]:
generate_data()

In [8]:
data = pd.read_json(path / "monte-carlo" / "data.json", orient="table")
data

,dim,delta_t,num_responses,num_signals,marginal_entropy
0,100,5.00,1,550,217.238096
1,100,5.00,1,550,216.086224
2,100,5.00,1,550,234.830855
3,100,5.00,1,550,226.434792
4,100,5.00,1,550,233.746046
...,...,...,...,...,...
2395,400,1.25,1,550,611.748674
2396,400,1.25,1,550,621.235770
2397,400,1.25,1,550,589.843572
2398,400,1.25,1,550,606.137892


In [9]:
estimates = data.groupby(['dim', 'delta_t']).agg(
    num_responses=('num_responses', 'sum'),
    num_signals=('num_signals', 'sum'),
    marginal_entropy=('marginal_entropy', 'mean'),
    stderr=('marginal_entropy', 'sem')
).reset_index()
estimates.to_json(path / "monte-carlo" / "estimates.json", orient="table")
estimates

,dim,delta_t,num_responses,num_signals,marginal_entropy,stderr
0,100,5.000000,400,220000,222.448826,0.361264
1,160,3.125000,400,220000,318.209109,0.447236
2,220,2.272727,400,220000,403.000345,0.528497
3,280,1.785714,400,220000,477.638080,0.594237
4,340,1.470588,400,220000,548.440902,0.681034
5,400,1.250000,400,220000,612.609541,0.743269
